In [1]:
import argparse
import random
from sampler import data_sampler
from config import Config
import torch
from model.bert_encoder import Bert_Encoder, Bert_LoRa
from model.dropout_layer import Dropout_Layer
from model.classifier import Softmax_Layer, Proto_Softmax_Layer
from data_loader import get_data_loader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.cluster import KMeans
import collections
from copy import deepcopy
from transformers import AutoTokenizer
import os
from process import sequence_data_sampler_bert_prompt, data_sampler_bert_prompt_deal_first_task
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from tqdm import tqdm
import logging
from peft import get_peft_model

2023-12-27 14:08:16.626452: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

class args:
    batch_size = 64
    gradient_accumulation_steps = 4
    total_round = 6
    drop_out = 0.5
    num_workers = 2
    step1_epochs = 10
    step2_epochs = 10
    num_protos = 1
    device = 'cuda'
    seed = 100
    max_grad_norm = 10
    task_length = 8
    kl_temp = 2
    temp = 0.1

    bert_path = 'bert-base-uncased'
    max_length = 256
    vocab_size = 30522
    marker_size = 4
    pattern = 'entity_marker'
    encoder_output_size = 768
    lora = True
    save_lora = './Checkpoint/LoRA_2'

    drop_p = 0.1
    f_pass = 10
    kappa_neg = 0.03
    kappa_pos = 0.05

    T_mult = 1
    rewarm_epoch_num = 2
    # StepLR
    decay_rate = 0.9
    decay_steps = 800
    task = 'tacred'
    shot = 10
    
config = args


config.device = torch.device(config.device)
config.n_gpu = torch.cuda.device_count()
config.batch_size_per_step = int(config.batch_size / config.gradient_accumulation_steps)

config.task = args.task
config.shot = args.shot
config.step1_epochs = 10
config.step2_epochs = 15
config.step3_epochs = 20
config.temperature = 0.08


config.relation_file = "/home/luungoc/Continual Learning/data/tacred/relation_name.txt"
config.rel_index = "/home/luungoc/Continual Learning/data/tacred/rel_index.npy"
config.rel_feature = "/home/luungoc/Continual Learning/data/tacred/rel_feature.npy"
config.num_of_relation = 41
config.rel_cluster_label = "/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_5/rel_cluster_label_0.npy"
config.training_file = "/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_5/train_0.txt"
config.valid_file = "/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_5/valid_0.txt"
config.test_file = "/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_5/test_0.txt"



In [3]:
for rou in range(1):
    test_cur = []
    test_total = []
    random.seed(100 + rou*10)
    sampler = data_sampler(config=config, seed=100 + rou*10)
    id2rel = sampler.id2rel
    rel2id = sampler.rel2id
    id2sentence = sampler.get_id2sent()
    

    # print(f"Param trainable: {sum(p.numel() for p in encoder.parameters() if p.requires_grad)}")
    
    num_class = len(sampler.id2rel)

    memorized_samples = {}
    memory = collections.defaultdict(list)
    history_relations = []
    history_data = []
    prev_relations = []
    classifier = None
    prev_classifier = None
    prev_encoder = None
    prev_dropout_layer = None
    relation_standard = {}
    forward_accs = []
    
        

[7 0 6 2 5 1 4 3]


In [4]:
import numpy as np

batch = 1
rel_index = np.load(config.rel_index)
rel_cluster_label = np.load(config.rel_cluster_label)
cluster_to_labels = {}
for index, i in enumerate(rel_index):
    if rel_cluster_label[index] in cluster_to_labels.keys():
        cluster_to_labels[rel_cluster_label[index]].append(i-1)
    else:
        cluster_to_labels[rel_cluster_label[index]] = [i-1]

task_length = 7
shuffle_index_old = list(range(task_length - 1))
random.shuffle(shuffle_index_old)
shuffle_index_old = np.argsort(shuffle_index_old)
shuffle_index = np.insert(shuffle_index_old, 0, task_length - 1)



indexs = cluster_to_labels[shuffle_index[batch]] 

In [5]:
indexs

[25, 15, 35, 37, 38]

In [47]:
for steps, (training_data, valid_data, test_data, current_relations, historic_test_data, seen_relations) in enumerate(sampler):
    print('---'*30 )
    total = 0
    for item in current_relations:
        total += len(training_data[item])
    # print('current training data num: ' + str(total))
    # print(f'length training data: {len(training_data)}')
    # print(f'length valid data: {len(valid_data)}')
    # print(f'length test data: {len(test_data)}')
    
    # print(f'current relation: {current_relations}')
    # print(f'length historic_test_data: {historic_test_data.keys()}')
    print(f'length seen relation: {len(test_data[current_relations[0]])}')
    # print(training_data[current_relations[0]][0])

------------------------------------------------------------------------------------------
length seen relation: 100
------------------------------------------------------------------------------------------
length seen relation: 9
------------------------------------------------------------------------------------------
length seen relation: 5
------------------------------------------------------------------------------------------
length seen relation: 52
------------------------------------------------------------------------------------------
length seen relation: 44
------------------------------------------------------------------------------------------
length seen relation: 37
------------------------------------------------------------------------------------------
length seen relation: 81
------------------------------------------------------------------------------------------
length seen relation: 100


In [7]:
samples = []
with open('/home/luungoc/Continual Learning/data/tacred/CFRLdata_10_100_10_5/test_0.txt') as file_in:
    for line in file_in:
        items = line.strip().split('\t')
        if (len(items[0]) > 0):
            relation_ix = int(items[0])
            if items[1] != 'noNegativeAnswer':
                candidate_ixs = [int(ix) for ix in items[1].split()]
                sentence = items[2].split('\n')[0]
                headent = items[3]
                headidx = [int(ix) for ix in items[4].split()]
                tailent = items[5]
                tailidx = [int(ix) for ix in items[6].split()]
                headid = items[7]
                tailid = items[8]
                samples.append([relation_ix, candidate_ixs, sentence, headent, headidx, tailent, tailidx, headid, tailid])
                